# Receipt printing in .NET (ESC/POS) with Czech diacritic

The basic idea is the same as the previous example, for printing using ESC/POS codes:

### Find available printers

We will need Windows printer names for the `RawPrint.NetStd` library. Instead of using the `System.Drawing.Common` package (which is fairly heavyweight), we can directly query the registry:

In [2]:
var printers = Microsoft.Win32.Registry.LocalMachine.OpenSubKey(@"SYSTEM\CurrentControlSet\Control\Print\Printers", false);
var printerNames = printers.GetSubKeyNames();

foreach (var name in printerNames) {
    Console.WriteLine(name);
}

Adobe PDF
Brother PT-H500
Canon iX6800 series
Cashino58 Printer
EPSON FX-890
Fax
Microsoft Print to PDF
Microsoft XPS Document Writer
OneNote (Desktop)
Snagit 2025
Snagit 25
XPrinter XP-58


### Using RawPrint.NetStd and ESCPOS

In [3]:
#r "nuget: RawPrint.NetStd"
#r "nuget: ESCPOS"

using ESCPOS;
using ESCPOS.Utils;

Installed Packages ESCPOS, 1.3.0 RawPrint.NetStd, 1.0.0

## Create receipt using ESC/POS mode

To make working with ESCPOS library easier, I created helper class `EscPosDocument`. It creates a memory stream with the data, which can be then printed later.

It also contains some utility methods for writing receipts.

There is notorious problem with characters with diacritics, as not every printer supports them. To overcome this problem in the most universal way, I just strip all diacritic marks from letters, because I'm lazy :)

In [6]:
using System.IO;

public class EscPosDocument {

    // Constructor

    public EscPosDocument() { 
        this.Write(Commands.InitializePrinter);
    }

    // Properties

    public Stream Stream { get; } = new MemoryStream();

    public int Columns { get; set; } = 32; // Typical values are 32 for 58 mm paper and 48 for 80 mm paper

    public float ReceiptTotal { get; private set; } = 0;

    public Func<string, byte[]> StringToBytes { get; init; } = RemoveDiacritic;

    // Basic write methods

    public void Write(byte[] bytes) => this.Stream.Write(bytes, 0, bytes.Length);

    public void Write(string text) => this.Write(this.StringToBytes(text));

    public void Write(params object[] items) {
        foreach (var item in items) {
            if (item is string s) {
                this.Write(s);
            } else if (item is byte[] b) {
                this.Write(b);
            } else {
                this.Write(item.ToString());
            }
        }
    }

    // Utility methods for writing lines

    public void WriteLine() => this.Write(Commands.LineFeed);

    public void WriteLine(string text) => this.Write(text, Commands.LineFeed);

    public void WriteLine(params object[] items) => this.Write(items.Append(Commands.LineFeed).ToArray());

    public void WriteDoubleAlignedLine(string left, string right) {
        var totalLength = left.Length + 1 + right.Length;
        if (totalLength > this.Columns) {
            // Too long, just write left and right on separate lines
            this.WriteLine(Commands.AlignToLeft, left);
            this.WriteLine(Commands.AlignToRight, right);
            this.Write(Commands.AlignToLeft);
        } else {
            // It will fit, so align left and right on the same line
            var spaces = new string(' ', this.Columns - left.Length - right.Length);
            this.WriteLine(Commands.AlignToLeft, left, spaces, right);
        }
    }

    public void WriteLines(int numberOfLines) {
        for (int i = 0; i < numberOfLines; i++) this.WriteLine();
    }

    // Other utility methods

    public void WriteHorizontalLine(char ch = '-') => this.WriteLine(new string(ch, this.Columns));

    //  Receipt specific methods

    public void WriteReceiptItem(string name, float unitPrice, int quantity) {
        var total = unitPrice * quantity;
        var priceInfo = quantity == 1 ? $"{unitPrice:N2} Kč" : $"{quantity} x {unitPrice:N2} Kč = {total:N2} Kč";
        this.ReceiptTotal += total;
        this.WriteDoubleAlignedLine(name, priceInfo);
    }

    public void WriteReceiptTotal() {
        this.Write(Commands.DoubleStrikeOn);
        this.WriteDoubleAlignedLine("Celkem", $"{this.ReceiptTotal:N2} Kč");
        this.Write(Commands.DoubleStrikeOff);
    }

    // Helper method to convert string to byte array and remove diacritics
    
    public static byte[] RemoveDiacritic(string text) {
        var normalized = text.Normalize(NormalizationForm.FormD);
        var sb = new System.Text.StringBuilder();
        foreach (var c in normalized) {
            var uc = System.Globalization.CharUnicodeInfo.GetUnicodeCategory(c);
            if (uc != System.Globalization.UnicodeCategory.NonSpacingMark) sb.Append(c);
        }
        var ascii = sb.ToString().Normalize(NormalizationForm.FormC);
        return System.Text.Encoding.ASCII.GetBytes(ascii);
    }

}

## Printing using Windows-1250 and ISO 8859-2

In the previous example we solved problem of Czech (and possibly other) diacritic mark by simply removing them. But maybe we can print them! It depends on the printer:

The printer usually supports several different code tables and character sets and you can select the code table to be used using ESC/POS sequence, usually `\x1b\x74` followed by number for the appropriate code page. If you get lucky, your printer supports your language and you just have to select the right code page and send data in the correct encoding.

If you aren't lucky, most printer allows you to define custom graphic forms for characters, so you can define the special characters you use. In my opinion, it usually isn't worth it.

It's throwback to old bad times, where we didn't had Unicode and had to work with code tables and character sets. I have yet to see receipt printer that supports UTF-8, though. It _may_ exist, but I believe it's something like mythical being.

I got lucky. My printer _Cashino PTP-II_ supports Czech diacritic in Windows-1250 charset. So I can just modify my code to print weird characters like `ř` or `ů`:

In [10]:
var doc = new EscPosDocument { 
    // Custom StringToBytes to use Windows-1250 encoding
    StringToBytes = text => {
        var encoding = System.Text.Encoding.GetEncoding("Windows-1250");
        return encoding.GetBytes(text);
        }
};

// Set code page to Windows-1250
doc.Write(Commands.SelectCodeTable((CodeTable)0x1e));; 

// Write header
doc.WriteLine(Commands.AlignToCenter, Commands.CharSizeDoubleHeight, "Altairis, s. r. o.", Commands.CharSizeReset);
doc.WriteLine("Bořivojova 35, 130 00 Praha 3");
doc.WriteLine("IČO: 27560911, DIČ: CZ27560911");
doc.WriteHorizontalLine('_');
doc.WriteLine();

// Write receipt items and total
doc.WriteReceiptItem("Položka 1", 49.00f, 1);
doc.WriteReceiptItem("Nějaký opravdu velmi dlouhý název položky 2", 199.00f, 2);
doc.WriteReceiptItem("Položka 3", 9.90f, 1);
doc.WriteReceiptItem("Položka 4", 25.00f, 5);
doc.WriteReceiptItem("Položka 5", 29.90f, 1);
doc.WriteHorizontalLine();
doc.WriteReceiptTotal();
doc.WriteLine();

// Write footer
doc.WriteLine(DateTime.Now.ToString("dd. MM. yyyy HH:mm"));
doc.Write(Commands.AlignToLeft);

// Add some blank lines at the end to eject the paper
doc.WriteLines(5);

// Send to printer
var prn = new RawPrint.NetStd.Printer();
prn.PrintRawStream("Cashino58 Printer", doc.Stream, documentName: "Test document", paused: false);

Actually, my printer supports two encodings usable for Czech characters, in addition to _Windows-1250_ it also supports _ISO 8859-2_ encoding. So the following example would print exactly the same data, only using different encoding:

In [9]:
var doc = new EscPosDocument { 
    // Custom StringToBytes to use ISO 8859-2 encoding
    StringToBytes = text => {
        var encoding = System.Text.Encoding.GetEncoding("ISO-8859-2");
        return encoding.GetBytes(text);
        }
};

// Set code page to ISO 8859-2
doc.Write(Commands.SelectCodeTable((CodeTable)0x24));

// Write header
doc.WriteLine(Commands.AlignToCenter, Commands.CharSizeDoubleHeight, "Altairis, s. r. o.", Commands.CharSizeReset);
doc.WriteLine("Bořivojova 35, 130 00 Praha 3");
doc.WriteLine("IČO: 27560911, DIČ: CZ27560911");
doc.WriteHorizontalLine('_');
doc.WriteLine();

// Write receipt items and total
doc.WriteReceiptItem("Položka 1", 49.00f, 1);
doc.WriteReceiptItem("Nějaký opravdu velmi dlouhý název položky 2", 199.00f, 2);
doc.WriteReceiptItem("Položka 3", 9.90f, 1);
doc.WriteReceiptItem("Položka 4", 25.00f, 5);
doc.WriteReceiptItem("Položka 5", 29.90f, 1);
doc.WriteHorizontalLine();
doc.WriteReceiptTotal();
doc.WriteLine();

// Write footer
doc.WriteLine(DateTime.Now.ToString("dd. MM. yyyy HH:mm"));
doc.Write(Commands.AlignToLeft);

// Add some blank lines at the end to eject the paper
doc.WriteLines(5);

// Send to printer
var prn = new RawPrint.NetStd.Printer();
prn.PrintRawStream("Cashino58 Printer", doc.Stream, documentName: "Test document", paused: false);

The ESCPOS library has the `Commands.SelectCodeTable` method, but it has just a limited set of code tables by default, neither of which suits my need. So that's why there is the weird casting of hexadecimal number to `CodeTable` enum.